In [54]:
import os
import pickle
import json
import glob
import pandas as pd
import numpy as np
import nilearn
from nilearn import datasets
from nilearn.masking import intersect_masks
from nilearn.input_data import NiftiLabelsMasker
from nilearn.connectome import ConnectivityMeasure

In [55]:

def calculate_functional_connectivity(parcellated_timeseries, connectivity_type):
    correlation_measure = ConnectivityMeasure(kind=connectivity_type)
    func_conn_matrix = correlation_measure.fit_transform([parcellated_timeseries])[0]
    return func_conn_matrix

def preprocess_confounds(confound_df):
    confound_df = confound_df.fillna(method='bfill')
    return confound_df


def get_timeseries(masker, img_path, confound_df):
    # parcellate timeseries
    parcellated_timeseries = masker.fit_transform(img_path, confounds=confound_df)
    
    return parcellated_timeseries

In [56]:
# inputs

data_dir = '../data_dir'
# which fmriprep dir to use
fmriprep_label = 'fmriprep'
# where the output for this script goes
output_dir = '../output'

# how to calculate connectivity matrix
# Options: {covariance, correlation, partial correlation, tangent, precision}
connectivity_type = 'correlation'

In [57]:
#Parcellation options
# where your parcellation mask is stored
parcellation_dir = '../parcellation_mask'

parc = 'smorgasbord' #parcellation type, for output filename

mask_filename = os.path.join(parcellation_dir, 'tpl-MNI152NLin2009cAsym_res-01_atlas-smorgasbord_dseg.nii.gz')
mask_labels = open(os.path.join(parcellation_dir, 'tpl-MNI152NLin2009cAsym_res-01_atlas-smorgasbord_dseg.txt'))


mask_labels = mask_labels.read().split('\n')
for idx, row in enumerate(mask_labels):
    mask_labels[idx] = mask_labels[idx].split('\t')

standardize = True
t_r = 1.0
low_pass = 0.125
high_pass = 0.042
detrend = False
masker = NiftiLabelsMasker(labels_img=mask_filename,
                           standardize=standardize,
                           low_pass=low_pass, high_pass=high_pass, t_r=t_r,
                           detrend=detrend,
                           memory='nilearn_cache', verbose=5)  


In [58]:
# extract time series for each task
bids_dir = os.path.join(data_dir, 'BIDS')
derivs_dir = os.path.join(bids_dir, 'derivatives')
fmriprep_dir = os.path.join(derivs_dir, fmriprep_label)

# multitask_BIDS/derivatives/fmriprep/[sub-01, sub-02, etc.]/[ses-1, ses-2, etc.]/func
fmriprep_sub_func_dir = os.path.join(fmriprep_dir, '*/*/func')

# first level objects: image, confounds, and events
sub_masks       = sorted(glob.glob(os.path.join(fmriprep_sub_func_dir,'*mask*.nii*')))
sub_confounds   = sorted(glob.glob(os.path.join(fmriprep_sub_func_dir,'*confounds*.tsv*')))
models_run_imgs = sorted(glob.glob(os.path.join(fmriprep_sub_func_dir,'*preproc*.nii*')))
sub_mask        = intersect_masks(sub_masks)
sub_confounds   = [pd.read_csv(f, delimiter='\t') for f in sub_confounds]

file_names = [os.path.basename(path) for path in models_run_imgs]
subject_names = [string.split("sub-")[1].split("_")[0] for string in file_names]
session_nums = [string.split("ses-")[1].split("_")[0] for string in file_names]
task_names = [string.split("task-")[1].split("_")[0] for string in file_names]
run_nums = [string.split("run-")[1].split("_")[0] for string in file_names]


# extract timeseries
for idx, _ in enumerate(file_names):
    confound_df = preprocess_confounds(sub_confounds[idx].copy())

    # get parcellated time series of the run (preprocessed by fmriprep), remove confounds
    parcellated_timeseries = get_timeseries(masker, models_run_imgs[idx], confound_df)

    func_conn_matrix = calculate_functional_connectivity(parcellated_timeseries, connectivity_type)

    # save
    parc_output_dir = os.path.join(output_dir, 'parcellated_timeseries')
    fc_output_dir = os.path.join(output_dir, 'functional_connectivity')
    parc_output_dir = os.path.join(parc_output_dir, f'sub-{subject_names[idx]}')
    fc_output_dir = os.path.join(fc_output_dir, f'sub-{subject_names[idx]}')

    if not os.path.exists(parc_output_dir):
        os.makedirs(parc_output_dir)
    if not os.path.exists(fc_output_dir):
        os.makedirs(fc_output_dir)

    fname = f'sub-{subject_names[idx]}_ses-{session_nums[idx]}_task-{task_names[idx]}_run-{run_nums[idx]}_parc-{parc}-timeseries.csv'

    save_path = os.path.join(parc_output_dir, fname)
    # pickle.dump(parcellated_timeseries, open(save_path, 'wb'))
    np.savetxt(save_path, parcellated_timeseries, delimiter=",")
    print('saved: ', save_path)
    print('(shape: ', parcellated_timeseries.shape, ')')

    fname = f'sub-{subject_names[idx]}_ses-{session_nums[idx]}_task-{task_names[idx]}_run-{run_nums[idx]}_parc-{parc}-FCM.csv'
    save_path = os.path.join(fc_output_dir, fname)
    # pickle.dump(func_conn_matrix, open(save_path, 'wb'))
    np.savetxt(save_path, func_conn_matrix, delimiter=",")
    print('saved: ', save_path)
    print('(shape: ', func_conn_matrix.shape, ')')

    print('from: ', models_run_imgs[idx])

print('done!')


hape': None}, confounds=[      global_signal  global_signal_derivative1  \
0      3917.747327                  -3.041453   
1      3914.705874                  -3.041453   
2      3918.193154                   3.487280   
3      3925.973967                   7.780814   
4      3936.832256                  10.858289   
..             ...                        ...   
325    3959.839409                  -0.754432   
326    3961.858823                   2.019414   
327    3962.413664                   0.554841   
328    3960.665770                  -1.747894   
329    3960.640964                  -0.024806   

     global_signal_derivative1_power2  global_signal_power2          csf  \
0                            9.25..., dtype=None, memory=Memory(location=nilearn_cache\joblib), memory_level=1, verbose=5)
[NiftiLabelsMasker.transform_single_imgs] Loading data from ..\data_dir\BIDS\derivatives\fmriprep\sub-s497\ses-1\func\sub-s497_ses-1_task-stroop_run-1_space-MNI152NLin2009cAsym_desc-prep

In [59]:

# with open(fname, 'rb') as f:
#     data = pickle.load(f)